In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM

In [2]:
class dataset:
    pass

df = pd.read_csv("multilabel_dataset.csv")
df_predicted = pd.read_csv("total_noanswer.csv")
df.head(5)

,Unnamed: 0,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,53,17,02/03/2018 09:27:39,395,1,1,31.0,47.0,31.0,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,1,443,6,02/03/2018 01:47:54,5835709,8,4,598.0,3427.0,342.0,...,20,350083.0,0.0,350083.0,350083.0,5485590.0,0.0,5485590.0,5485590.0,Benign
2,2,3389,6,02/03/2018 03:43:39,3888916,14,8,1442.0,1731.0,725.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,3,80,6,02/03/2018 05:29:44,853,3,4,161.0,488.0,161.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,4,50966,6,02/03/2018 10:11:07,57,2,0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [3]:
df = df.drop(columns='Unnamed: 0')

In [5]:
df.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1623293 entries, 0 to 1623292
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1623293 non-null  int64  
 1   Protocol           1623293 non-null  int64  
 2   Timestamp          1623293 non-null  object 
 3   Flow Duration      1623293 non-null  int64  
 4   Tot Fwd Pkts       1623293 non-null  int64  
 5   Tot Bwd Pkts       1623293 non-null  int64  
 6   TotLen Fwd Pkts    1623293 non-null  float64
 7   TotLen Bwd Pkts    1623293 non-null  float64
 8   Fwd Pkt Len Max    1623293 non-null  float64
 9   Fwd Pkt Len Min    1623293 non-null  float64
 10  Fwd Pkt Len Mean   1623293 non-null  float64
 11  Fwd Pkt Len Std    1623293 non-null  float64
 12  Bwd Pkt Len Max    1623293 non-null  float64
 13  Bwd Pkt Len Min    1623293 non-null  float64
 14  Bwd Pkt Len Mean   1623293 non-null  float64
 15  Bwd Pkt Len Std    1623293 non-n

In [7]:
df.shape

(1623293, 80)

In [9]:
# Find inf, NaN, All 0
# inf, NaN : 'Flow Byts/s', 'Flow Pkts/s'
# All 0 : 'Bwd PSH Flags','Fwd URG Flags','Bwd URG Flags','CWE Flag Count','Fwd Byts/b Avg','Fwd Pkts/b Avg','Fwd Blk Rate Avg','Bwd Byts/b Avg','Bwd Pkts/b Avg','Bwd Blk Rate Avg'
df.describe()


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,...,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06,1.623293e+06
mean,9.185777e+03,8.753311e+00,1.218434e+07,2.435085e+01,6.184580e+00,9.848913e+02,4.540388e+03,2.003709e+02,1.108099e+01,5.028538e+01,...,2.073935e+01,1.799534e+01,1.731388e+05,8.791442e+04,2.639134e+05,1.147115e+05,4.803805e+06,1.507785e+05,4.932766e+06,4.661911e+06
std,1.894261e+04,4.919311e+00,3.088014e+07,1.590507e+03,1.508257e+02,5.140297e+04,2.174271e+05,3.025947e+02,2.448549e+01,6.086769e+01,...,1.589535e+03,7.693534e+00,2.489933e+06,1.532713e+06,3.326034e+06,2.085602e+06,1.590636e+07,1.660842e+06,1.615778e+07,1.578273e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,6.000000e+00,4.900000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,6.000000e+00,2.097300e+04,2.000000e+00,1.000000e+00,4.300000e+01,1.010000e+02,4.000000e+01,0.000000e+00,3.600000e+01,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.389000e+03,1.700000e+01,2.990876e+06,5.000000e+00,4.000000e+00,4.550000e+02,5.820000e+02,3.020000e+02,2.900000e+01,6.740000e+01,...,1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553400e+04,1.700000e+01,1.200000e+08,3.096290e+05,2.656600e+04,9.908128e+06,3.867802e+07,4.152800e+04,1.460000e+03,1.203754e+04,...,3.096280e+05,4.400000e+01,1.113114e+08,7.497829e+07,1.113114e+08,1.113114e+08,1.200000e+08,7.493599e+07,1.200000e+08,1.200000e+08


In [10]:
df_predicted.describe()

,dst_port,protocol,flow_duration,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,...,fwd_act_data_pkts,fwd_seg_size_min,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
count,12501.000000,12501.0,1.250100e+04,12501.000000,12501.0,1.250100e+04,12501.0,12501.000000,12501.000000,12501.000000,...,12501.00000,12501.0,1.250100e+04,1.250100e+04,1.250100e+04,1.250100e+04,1.250100e+04,1.250100e+04,1.250100e+04,1.250100e+04
mean,3715.477802,6.0,2.447975e+07,7.016879,0.0,6.197359e+03,0.0,373.742261,60.501240,119.596010,...,5.62603,20.0,4.570678e+04,4.642705e+04,9.995754e+04,1.543567e+03,9.100694e+06,1.034236e+07,2.144696e+07,4.519078e+04
std,11906.791120,0.0,1.047696e+07,76.907398,0.0,1.833340e+05,0.0,1692.337233,1.662943,168.204841,...,76.83863,0.0,5.247163e+05,5.257001e+05,1.107267e+06,4.150912e+04,4.945828e+06,4.821658e+06,9.671536e+06,1.904658e+05
min,80.000000,6.0,0.000000e+00,1.000000,0.0,6.000000e+01,0.0,60.000000,60.000000,60.000000,...,0.00000,20.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,80.000000,6.0,2.716014e+07,4.000000,0.0,2.460000e+02,0.0,66.000000,60.000000,61.500000,...,3.00000,20.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.305678e+06,9.551115e+06,2.177167e+07,9.084000e+03
50%,80.000000,6.0,2.919319e+07,5.000000,0.0,6.120000e+02,0.0,355.000000,60.000000,117.400000,...,4.00000,20.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.150338e+06,1.113120e+07,2.372316e+07,1.059900e+04
75%,80.000000,6.0,3.002376e+07,5.000000,0.0,7.540000e+02,0.0,481.000000,60.000000,144.666667,...,4.00000,20.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.499955e+07,1.498582e+07,2.998566e+07,1.606100e+04
max,59360.000000,6.0,6.336587e+07,4786.000000,0.0,8.437957e+06,0.0,65302.000000,74.000000,5662.657188,...,4782.00000,20.0,1.325843e+07,1.359454e+07,3.150764e+07,2.402712e+06,1.723743e+07,1.722575e+07,3.446318e+07,4.967589e+06


In [11]:
df.dtypes

Dst Port           int64
Protocol           int64
Timestamp         object
Flow Duration      int64
Tot Fwd Pkts       int64
                  ...   
Idle Mean        float64
Idle Std         float64
Idle Max         float64
Idle Min         float64
Label             object
Length: 80, dtype: object

In [12]:
df['Label'].unique()

array(['Benign', 'Bot', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DoS attacks-GoldenEye', 'DoS attacks-Slowloris',
       'DoS attacks-Hulk', 'DoS attacks-SlowHTTPTest',
       'DDoS attacks-LOIC-HTTP', 'DDOS attack-HOIC',
       'DDOS attack-LOIC-UDP', 'Brute Force -Web', 'Brute Force -XSS',
       'SQL Injection', 'Infilteration'], dtype=object)

In [13]:
df.isnull()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623288,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1623289,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1623290,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1623291,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
pd.Categorical(df['Flow Byts/s'])

C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


[197468.354400, 689.719107, 815.908598, 760844.079700, 0.000000, ..., 0.0000, 0.0000, 181347.1503, 227743.2712, 189618.6441]
Length: 1623293
Categories (851716, float64): [0.000000, 0.008432, 0.008704, 0.008724, ..., 7.880000e+08, 8.494545e+08, 8.860000e+08, inf]

In [15]:
pd.Categorical(df['Flow Pkts/s'])# to check different value

C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


[5063.291139, 2.056305, 5.657103, 8206.330598, 35087.719300, ..., 23.442536, 7547.169811, 2072.538860, 4140.786749, 2118.644068]
Length: 1623293
Categories (833119, float64): [0.016667, 0.016668, 0.016668, 0.016669, ..., 2500000.0, 3000000.0, 4000000.0, inf]

In [16]:
df['Flow Pkts/s'].unique()

array([5.06329114e+03, 2.05630541e+00, 5.65710342e+00, ...,
       2.92645811e+01, 8.65989309e+00, 2.34425365e+01])

In [17]:
# Drop timestamp
df.drop('Timestamp',axis=1,inplace=True)
df_predicted.drop('timestamp',axis=1,inplace=True)

In [18]:
df.shape

(1623293, 79)

In [19]:
df.drop(['Bwd PSH Flags','Fwd URG Flags','Bwd URG Flags','CWE Flag Count','Fwd Byts/b Avg','Fwd Pkts/b Avg','Fwd Blk Rate Avg','Bwd Byts/b Avg','Bwd Pkts/b Avg','Bwd Blk Rate Avg'],axis=1,inplace=True)
df_predicted.drop(['bwd_psh_flags','fwd_urg_flags','bwd_urg_flags','cwe_flag_count','fwd_byts_b_avg','fwd_pkts_b_avg','fwd_blk_rate_avg','bwd_byts_b_avg','bwd_pkts_b_avg','bwd_blk_rate_avg'],axis=1,inplace=True)

In [20]:
df.shape

(1623293, 69)

In [21]:
# check NULL
df.isnull().sum()

Dst Port         0
Protocol         0
Flow Duration    0
Tot Fwd Pkts     0
Tot Bwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle Min         0
Label            0
Length: 69, dtype: int64

In [22]:
# removing NAN and INF values
df['Flow Pkts/s']= df['Flow Pkts/s'].fillna(0)
df['Flow Byts/s']= df['Flow Byts/s'].fillna(0)
df['Flow Pkts/s']= df['Flow Pkts/s'].replace(np.Infinity,1)
df['Flow Byts/s']= df['Flow Byts/s'].replace(np.Infinity,1)

df_predicted['flow_pkts_s']= df_predicted['flow_pkts_s'].fillna(0)
df_predicted['flow_byts_s']= df_predicted['flow_byts_s'].fillna(0)
df_predicted['flow_pkts_s']= df_predicted['flow_pkts_s'].replace(np.Infinity,1)
df_predicted['flow_byts_s']= df_predicted['flow_byts_s'].replace(np.Infinity,1)

In [23]:
# now no INF and NAN values
pd.Categorical(df['Flow Byts/s'])

C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


[197468.354400, 689.719107, 815.908598, 760844.079700, 0.000000, ..., 0.0000, 0.0000, 181347.1503, 227743.2712, 189618.6441]
Length: 1623293
Categories (851716, float64): [0.000000, 0.008432, 0.008704, 0.008724, ..., 7.300000e+08, 7.880000e+08, 8.494545e+08, 8.860000e+08]

In [24]:
pd.Categorical(df['Flow Pkts/s'])

C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
C:\Users\gmrdk\anaconda3\envs\ml-ids\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


[5063.291139, 2.056305, 5.657103, 8206.330598, 35087.719300, ..., 23.442536, 7547.169811, 2072.538860, 4140.786749, 2118.644068]
Length: 1623293
Categories (833119, float64): [0.016667, 0.016668, 0.016668, 0.016669, ..., 2000000.0, 2500000.0, 3000000.0, 4000000.0]

In [25]:
df.shape

(1623293, 69)

In [26]:
#dataset.train = df.groupby('Label').apply(pd.DataFrame.sample, frac=0.8).reset_index(level='Label', drop=True)
#dataset.test = df.drop(dataset.train.index)
#dataset.label = dataset.train.Label.copy()

In [ ]:
dataset.label.unique()

In [32]:
d1 = df.replace('Benign', 0) # here converting the categorical values to numeric
d2 = d1.replace('Bot', 1)
d3 = d2.replace('Brute Force -Web', 2)
d4 = d3.replace('Brute Force -XSS', 3)
d5 = d4.replace('DDOS attack-HOIC', 4)
d6 = d5.replace('DDOS attack-LOIC-UDP', 5)
d7 = d6.replace('DDoS attacks-LOIC-HTTP' ,6)
d8 = d7.replace('DoS attacks-GoldenEye', 7)
d9 = d8.replace('DoS attacks-Hulk', 8)
d10 = d9.replace('DoS attacks-SlowHTTPTest', 9)
d11 = d10.replace('DoS attacks-Slowloris', 10)
d12 = d11.replace('FTP-BruteForce', 11)
d13 = d12.replace('Infilteration', 12)
d14 = d13.replace('SQL Injection', 13)
d15 = d14.replace('SSH-Bruteforce', 14)
d15_label = d15.Label.copy()
d15_label.unique()
d15_label.value_counts()

0     1348471
4       68601
6       57619
8       46191
1       28619
11      19336
14      18759
12      16193
9       13989
7        4151
10       1099
5         173
2          61
3          23
13          8
Name: Label, dtype: int64

In [38]:
d15 = d15.drop(columns="Label")

In [39]:
df_predicted.columns = d15.columns

In [30]:
d15_label.unique()

array([ 0,  1, 11, 14,  7, 10,  8,  9,  6,  4,  5,  2,  3, 13, 12],
      dtype=int64)

In [ ]:
df.unique() 

In [ ]:
"""
dataset.test_label = dataset.test.Label.copy()
dataset.test_label.unique()
a1_label = dataset.test.Label.copy()
a1_label.unique()
"""

In [ ]:
"""
a1 = dataset.test.replace('Benign', 0)
a2 = a1.replace('Bot', 1)
a5_label = a2.Label.copy()
a5_label.unique()
a5_label.value_counts()
"""

In [40]:
# Categorizing Protocol -> Protocol_0, Protocol_6, Protocol_17
category_variables = ["Protocol"]
for cv in category_variables:
    d15[cv] = d15[cv].astype("category")
    df_predicted[cv] = df_predicted[cv].astype("category")
    
    print("Length of Categories for {} are {}".format(cv , len(d15[cv].cat.categories)))
    print("Categories for {} are {} \n".format(cv ,df_predicted[cv].cat.categories))

Length of Categories for Protocol are 3
Categories for Protocol are Int64Index([6], dtype='int64') 



In [69]:
dummy_variables_2labels = category_variables
    
class preprocessing:
    train_labels = pd.get_dummies(d15, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)
    test_labels = pd.get_dummies(df_predicted, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)

In [70]:
preprocessing.test_labels.insert(67,"Protocol_0",0,True)
preprocessing.test_labels.insert(69,"Protocol_17",0,True)
preprocessing.test_labels.head()

,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol_0,Protocol_6,Protocol_17
0,80,26870384,5,0,659,0,413,60,131.8,140.619202,...,0.0,0,0,7291957.667,10295440.45,21851909,9971,0,1,0
1,43908,372198,5,0,486,0,214,66,97.2,58.482134,...,0.0,0,0,157598.500,158.50,157757,157440,0,1,0
2,80,27580343,5,0,827,0,581,60,165.4,207.812993,...,0.0,0,0,7517379.667,10594394.43,22500101,7682,0,1,0
3,80,27248690,5,0,627,0,381,60,125.4,127.821125,...,0.0,0,0,7409746.000,10452004.13,22191106,7534,0,1,0
4,80,28830565,5,0,791,0,545,60,158.2,193.413960,...,0.0,0,0,7927495.000,11165319.06,23717619,9795,0,1,0


In [71]:
preprocessing.train_labels.head()

,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol_0,Protocol_6,Protocol_17
0,53,395,1,1,31.0,47.0,31.0,31.0,31.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
1,443,5835709,8,4,598.0,3427.0,342.0,0.0,74.750000,128.886162,...,0.0,350083.0,350083.0,5485590.0,0.0,5485590.0,5485590.0,0,1,0
2,3389,3888916,14,8,1442.0,1731.0,725.0,0.0,103.000000,191.125412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
3,80,853,3,4,161.0,488.0,161.0,0.0,53.666667,92.953393,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
4,50966,57,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0


In [73]:
preprocessing.train_labels.to_csv("preprocessed_train_multi.csv")
preprocessing.test_labels.to_csv("preprocessed_test_multi.csv")